# Table of Contents
 <p>

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from IPython.display import display, HTML, JSON
from vega import Vega
from vega3 import Vega as Vega3
import requests
import json
import re
%reload_ext version_information
%version_information vega, vega3, IPython, ipywidgets, re

Software versions
Python 3.6.5 64bit [GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]
IPython 6.2.1
OS Darwin 16.7.0 x86_64 i386 64bit
vega 0.5.0
vega3 0.5.0
IPython 6.2.1
ipywidgets 7.2.1
re 2.2.1
Fri Jun 08 12:26:13 2018 CEST

In [3]:
def patchWidget(widgetId, datasetId, body):
    return

def formatWidget2read(widgetConfig):
    object2format= [
       {'pattern':r"({{([^}}]*)(year)}})", 'sub':'2020'},
       {'pattern':r"({{([^}}]*)(water_column)}})", 'sub':'ws2028tl'},
       {'pattern':r"({{([^}}]*)(crop)}})", 'sub':'maize'},
        {'pattern':r"({{([^}}]*)(commodity)}})", 'sub':'maize'},
       {'pattern':r"({{([^}}]*)(iso)}})", 'sub':'ESP'},
        {'pattern':r"({{([^}}]*)(countryName)}})", 'sub':'Spain'}
                   ]
    
    n=json.dumps(widgetConfig)
    for paterns in object2format:
        n = re.compile(paterns['pattern']).sub(paterns['sub'], n)
    return json.loads(n)



In [4]:
wApp=widgets.Dropdown(
    options={'Resource Watch': 'rw', 'Aqueduct': 'aqueduct', 'Prep': 'prep'},
    value='aqueduct',
    description='Choose Aaplication:',
)
wsize=widgets.IntSlider(min=1, max=1e3, step=1,continuous_update=False, value=1000)

wWidgets=widgets.Dropdown(
    description='Choose widgets:',
)
box_layout = Layout(display='flex',
                    flex_flow='column',
                    height='500px',
                    width='auto')
wCodeBox= widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Widget:',
    disabled=False,
    layout=box_layout
)
wCodeBox2= widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Widget Id:',
    disabled=False
)
wCodeBox3= widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Dataset Id:',
    disabled=False
)


def f(x, y, z, w):
    widgetsUrl="https://api.resourcewatch.org/v1/widget?app={app}&page[size]={size}".format(app=x, size=y)
    
    r=requests.get(widgetsUrl)
    if r.status_code == 200:
        opt = {widget['attributes']['name']: widget['id'] for widget in r.json()['data']}
        wWidgets.options = opt
        
        widgetUrl="https://api.resourcewatch.org/v1/widget/{widgetId}".format(widgetId=z)
        s=requests.get(widgetUrl)
        if s.status_code == 200:
            mywidget = s.json()['data']['attributes']['widgetConfig']
            mywidget['width']=300
            mywidget['height']=300
            if x=='aqueduct':
                display(Vega(formatWidget2read(mywidget)))
                wCodeBox.value = json.dumps(mywidget)
                wCodeBox2.value = z
                wCodeBox3.value = s.json()['data']['attributes']['dataset']
            else:
                
                display(Vega3(mywidget))
                   

out = widgets.interactive_output(f, {'x': wApp, 'y':wsize, 'z':wWidgets, 'w':wCodeBox})
ui = widgets.VBox([widgets.HBox([wApp,wsize]),wWidgets,widgets.HBox([wCodeBox,widgets.VBox([wCodeBox3,wCodeBox2]),out])])
display(ui)
#out.observe(handle_slider_change, names='All')
